In [5]:
import pandas as pd
import os

In [12]:
path = "Data/"

crypto ={}

for file in os.listdir(path):
    if file.endswith('.csv'):
        file_path = os.path.join(path, file)

        df = pd.read_csv(file_path)

        crypto_name = file.split('_')[1].split('.')[0]

        crypto[crypto_name] = df

In [13]:
crypto

{'NEM':        SNo Name Symbol                 Date      High       Low      Open  \
 0        1  NEM    XEM  2015-04-02 23:59:59  0.000323  0.000227  0.000242   
 1        2  NEM    XEM  2015-04-03 23:59:59  0.000330  0.000291  0.000309   
 2        3  NEM    XEM  2015-04-04 23:59:59  0.000318  0.000251  0.000310   
 3        4  NEM    XEM  2015-04-05 23:59:59  0.000283  0.000218  0.000272   
 4        5  NEM    XEM  2015-04-06 23:59:59  0.000299  0.000229  0.000232   
 ...    ...  ...    ...                  ...       ...       ...       ...   
 2283  2284  NEM    XEM  2021-07-02 23:59:59  0.131455  0.123406  0.128181   
 2284  2285  NEM    XEM  2021-07-03 23:59:59  0.136359  0.128350  0.131542   
 2285  2286  NEM    XEM  2021-07-04 23:59:59  0.140061  0.129912  0.134751   
 2286  2287  NEM    XEM  2021-07-05 23:59:59  0.137633  0.126915  0.137633   
 2287  2288  NEM    XEM  2021-07-06 23:59:59  0.136318  0.128804  0.130591   
 
          Close        Volume     Marketcap  
 0     0.